# Prepration
Loading env variables and vectorDB

In [37]:
from dotenv import load_dotenv
import os
import pickle

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("API_KEY")

In [38]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

## Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [39]:
from langchain.agents import load_tools
from langchain.llms import OpenAI

llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=API_KEY)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion:

In [41]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name = "Math Tool",
        func=tools[0].run,
        description="Tool to calculate, nothing else"
    )
]

In [42]:
from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)
agent.run("How are you?")



> Entering new AgentExecutor chain...
 This question is not a math problem.
Action: None
Action Input: N/A
Observation: None is not a valid tool, try another one.
Thought: I should use the Math Tool to answer this question.
Action: Math Tool
Action Input: "I'm doing great!"

ValueError: unknown format from LLM: This is not a mathematical expression, so it cannot be executed using Python's numexpr library.

In [43]:
agent.run("What is 100 devided by 25?")



> Entering new AgentExecutor chain...
 I need to use a tool to calculate this
Action: Math Tool
Action Input: 100/25
Observation: Answer: 4.0
Thought: I now know the final answer
Final Answer: 100 divided by 25 is equal to 4.0.

> Finished chain.


'100 divided by 25 is equal to 4.0.'

# Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool.

In [48]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "customer service search"
    description = """useful for when you need to answer questions about billing, shipping, orders, sales opportunity
    The question is about one of these things 
    output should be the answer to the question."""

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [49]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [51]:
agent.run("My feet are bleeding from walking barefoot,I am waiting for my shoes to arrive") 



> Entering new AgentExecutor chain...
 I need to look at the status of my order
Action: restaurant search
Action Input: my order status
Observation: do you know if my package shipped,Shipping Info
has my order been fulfilled,Shipping Info
when can I see the order status for my purchase?,Shipping Info
what is my shipping status?,Shipping Info
do you have my tracking # handy?,Shipping Info
"my shoes haven't arrived, where are they?",Shipping Info
"my order says in progress, what does that mean?",Shipping Info
is there a expected delivery date for my latest order,Shipping Info
when should I receive my items by,Shipping Info
can I get an update on my order status?,Shipping Info
who is the shipper of my products? I can't find the tracking info,Shipping Info
are those shoes being shipped overnight?,Shipping Info
Do you do overnight shipping on all orders,Shipping Info
what is the timeline to receive my order?,Shipping Info
what is the shipping method,Shipping Info
Has my order been sent?,S

'You should contact the restaurant to see if you can make changes to your order. Additionally, you can check the Shipping Info, Order Change, and Sales Opportunity sections on the restaurant search to answer any questions you may have about your order, shipping, and sales opportunities.'